In [ ]:
# Training CC on Extended Yale face Dataset

In [ ]:
import os
import numpy as np
import torch
import torchvision
import argparse
from modules import transform, resnet, network, contrastive_loss
from utils import yaml_config_hook, save_model
from torch.utils import data

In [ ]:
def train():
    loss_epoch = 0
    for step, ((x_i, x_j), _) in enumerate(data_loader):
        optimizer.zero_grad()
        x_i = x_i.to('cuda')
        x_j = x_j.to('cuda')
        z_i, z_j, c_i, c_j = model(x_i, x_j)
        loss_instance = criterion_instance(z_i, z_j)
        loss_cluster = criterion_cluster(c_i, c_j)
        loss = loss_instance + loss_cluster
        loss.backward()
        optimizer.step()
        if step % 50 == 0:
            print(
                f"Step [{step}/{len(data_loader)}]\t loss_instance: {loss_instance.item()}\t loss_cluster: {loss_cluster.item()}")
        loss_epoch += loss.item()
    return loss_epoch

In [ ]:
parser = argparse.ArgumentParser()
config = yaml_config_hook("config/config.yaml")
for k, v in config.items():
    parser.add_argument(f"--{k}", default=v, type=type(v))
args = parser.parse_args([])

if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)

torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)

In [ ]:
args.model_path = 'save/YalefaceB'
args.image_size = (192,168)

In [ ]:
args

In [ ]:
dataset = torchvision.datasets.ImageFolder(
    root='./dataset/ExtendedYaleB',
    transform=transform.Transforms(size=(192,168), blur=True),
)

class_num = len(dataset.classes)

In [ ]:
args.batch_size = 128

In [ ]:
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=args.batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=args.workers,
)

In [ ]:
res = resnet.get_resnet(args.resnet)
model = network.Network(res, args.feature_dim, class_num)
model = model.to('cuda')

In [ ]:
# optimizer / loss
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

In [ ]:
loss_device = torch.device("cuda")
criterion_instance = contrastive_loss.InstanceLoss(args.batch_size, args.instance_temperature, loss_device).to(
    loss_device)
criterion_cluster = contrastive_loss.ClusterLoss(class_num, args.cluster_temperature, loss_device).to(loss_device)

In [ ]:
import sys

# Holding the original output object. i.e. console out
orig_stdout = sys.stdout

# Opening the file to write file deletion logs.
f = open('outgan_train_yale_cc.txt', 'a+')

# Changing standard out to file out. 
sys.stdout = f
# This will write to the file. 
print("xyz") 

In [ ]:
# train
for epoch in range(args.start_epoch, args.epochs):
    lr = optimizer.param_groups[0]["lr"]
    loss_epoch = train()
    if epoch % 10 == 0:
        save_model(args.model_path, model, optimizer, epoch, 'yale_face')
    print(f"Epoch [{epoch}/{args.epochs}]\t Loss: {loss_epoch / len(data_loader)}")
save_model(args, model, optimizer, args.epochs)

In [ ]:
# Closing the file.
f.close()


# replacing the original output format to stdout.
sys.stdout = orig_stdout

# This will print onto the console.
print("xyz") 